In [ ]:
### 只做单模型，其他的内容，等等再说，现在把单模型的表现提升上去再说吧
### 测试看下来，单纯得跑模型似乎会应该Y值的方差而产生巨大问题，我们需要重新做一次分类，明天的核心任务吧

In [1]:
import sys
sys.path.append("../")
import pandas as pd
import numpy as np
from dateutil.parser import parse
from sklearn import metrics
import datetime

import model_ml as mm
import feat_engineering as fe
import feat_selection as fs
from param_config import config

In [12]:
dfTrain = pd.read_csv(config.original_train_data_path)
dfPred = pd.read_csv(config.original_pred_data_path)
predictors = dfPred.columns.tolist()[4:]
check_missing = ['PartI_1','PartII_1','PartIII_1','PartIV_1','PartV_1','PartVI_1']


###清理异常Y值
dfTrain = dfTrain.loc[dfTrain['Y']<dfTrain['Y'].max()]
dfTrain = dfTrain.reset_index(drop=True)

dfTrain = dfTrain.loc[dfTrain['Y']>dfTrain['Y'].quantile(0.9)]

In [13]:
def FeatAll(train,pred):
    dfAll = pd.concat([train,pred])
    dfAll = dfAll.reset_index(drop=True)
    dfAll['date'] = (pd.to_datetime(dfAll['date']) - parse('2017-10-09')).dt.days

    ###这个部分将每一列分箱，并且计算基于该列分类的其他列的百分比顺序，目前看来没啥用
    var_dict = {}
    for colBase in predictors+['sex','age']:
        if dfAll[colBase].nunique()<=5:
            var_dict[colBase] = [x for x in predictors if x!=colBase]
        else:
            dfAll[colBase+'_based'] = pd.cut(dfAll[colBase],bins=5,labels=[colBase+'_bin_%d'%i for i in range(5)])
            dfAll[colBase+'_based'] = dfAll[colBase+'_based'].astype(str)
            dfAll[colBase+'_based'].fillna(colBase+'_bin_NA',inplace = True)
            var_dict[colBase+'_based'] = [x for x in predictors if x!=colBase]
    #dfAllPcentByOther = fe.pcent_by_other_col(dfAll,var_dict,['ID']) 
    #dfAllPcentByOther = pd.read_csv('../../Cache/pcent_by_other.csv')
    
    for var in dfAll.columns:
        if '_based' in var:
            del dfAll[var]
            
    ###根据日期放置一个百分比顺序作为尝试
    dfAllPcentByDate = fe.pcent_by_other_col(dfAll,{'date':predictors},['ID']) 
    
    '''dfAll['date'] = pd.qcut(dfAll['date'],q=10,labels=['date_bin_%d'%i for i in range(10)])
    dfAll = pd.concat([dfAll,pd.get_dummies(dfAll['date'])],axis=1)'''
    ###目前不想使用日期，觉得用处不大
    del dfAll['date']

    #for minus in 
    
    for ratio in [['PartI_6','PartI_5'],['PartI_7','PartI_5'],['PartII_3','PartII_2'],['PartII_4','PartII_2'],['PartV_2','PartV_1'],['PartII_1','PartII_2']]:
        dfAll[ratio[0]+'_divided_'+ratio[1]] = dfAll[ratio[0]]/dfAll[ratio[1]]
        
    for multiply in [['PartV_1','PartV_9'],['PartV_1','PartV_10'],['PartV_1','PartV_11'],['PartV_1','PartV_12'],['PartV_1','PartV_13']]:
        dfAll[multiply[0]+'_multiply_'+multiply[1]] = dfAll[multiply[0]]/dfAll[multiply[1]]
    
    
    ###看看missing 的情况
    dfAll['missing_cnt'] = dfAll[check_missing].isnull().sum(axis=1)
    for i in range(len(check_missing)):
        dfAll['missing_part%d'%i] = dfAll[check_missing[i]].isnull().astype(int)
    
    
    '''for key,group in {'PartI':['PartI_1','PartI_2','PartI_3','PartI_4'],'PartII':['PartII_1','PartII_3','PartII_4']}.items():
        dfAll['group_'+key+'_std'] = dfAll[group].std(axis=1)
        dfAll['group_'+key+'_mean'] = dfAll[group].mean(axis=1)
        dfAll['group_'+key+'_median'] = dfAll[group].median(axis=1)'''
    
    #dfAll = dfAll.merge(dfAllPcentByOther,'inner','ID')
    dfAll = dfAll.merge(dfAllPcentByDate,'inner','ID')
    
    
    dfTrain = dfAll.loc[dfAll['ID'].isin(train['ID'])]
    dfPred = dfAll.loc[dfAll['ID'].isin(pred['ID'])]
    
    return dfTrain,dfPred,var_dict

In [14]:
dfTrain,dfPred,var_dict = FeatAll(dfTrain,dfPred)
predictors = dfPred.columns.tolist()
predictors.remove('ID')
predictors.remove('Y')

C:\Users\Leo Mao\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [15]:
len(predictors)

94

In [16]:
'''keep = ['ID']
for var in predictors:
    if 'pcent' in var and '_y' not in var:
        print(var)
        keep.append(var)
keep
dfAll = pd.concat([dfTrain[keep],dfPred[keep]])
rename_dict = {}
for var in keep:
    if '_x' in var:
        rename_dict[var] = var[0:-2]
dfAll = dfAll.rename(columns=rename_dict)
dfAll.to_csv('../../Cache/pcent_by_other.csv',index = False)'''

"keep = ['ID']\nfor var in predictors:\n    if 'pcent' in var and '_y' not in var:\n        print(var)\n        keep.append(var)\nkeep\ndfAll = pd.concat([dfTrain[keep],dfPred[keep]])\nrename_dict = {}\nfor var in keep:\n    if '_x' in var:\n        rename_dict[var] = var[0:-2]\ndfAll = dfAll.rename(columns=rename_dict)\ndfAll.to_csv('../../Cache/pcent_by_other.csv',index = False)"

In [17]:
n_splits=5
test_result,result,imp = mm.xgb_kfold(dfTrain,dfPred,predictors,n_splits)

[0]	train-rmse:8.8561	eval-rmse:8.71965
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 10 rounds.
[100]	train-rmse:3.87061	eval-rmse:3.97918
[200]	train-rmse:2.28375	eval-rmse:2.8346
[300]	train-rmse:1.79374	eval-rmse:2.67881
Stopping. Best iteration:
[326]	train-rmse:1.72219	eval-rmse:2.67673

Best tree is 327, performance is 1.722187, 2.676727
[0]	train-rmse:8.69484	eval-rmse:9.35785
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 10 rounds.
[100]	train-rmse:3.80603	eval-rmse:4.65414
[200]	train-rmse:2.26043	eval-rmse:3.30007
[300]	train-rmse:1.79767	eval-rmse:2.9558
[400]	train-rmse:1.5842	eval-rmse:2.86137
[500]	train-rmse:1.44296	eval-rmse:2.8239
[600]	train-rmse:1.32429	eval-rmse:2.8085
Stopping. Best iteration:
[613]	train-rmse:1.31102	eval-rmse:2.80586

Best tree is 614, performance is 1.311023, 2.805860
[0]	tra

In [18]:
for i in range(1,n_splits+1):
    imp['imp_fold%d'%i] = imp['imp_fold%d'%i]/imp['imp_fold%d'%i].sum()
imp['sum_imp'] = imp[['imp_fold%d'%i for i in range(1,n_splits+1)]].sum(axis=1)

In [19]:
imp.sort_values('sum_imp',ascending=False)
tmpPredictor = imp.sort_values('sum_imp',ascending=False)['variable'].values.tolist()[0:40]

In [20]:
test_result,result,imp = mm.xgb_kfold(dfTrain,dfPred,tmpPredictor,n_splits)

[0]	train-rmse:8.85632	eval-rmse:8.71935
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 10 rounds.
[100]	train-rmse:3.88836	eval-rmse:3.98554
[200]	train-rmse:2.29915	eval-rmse:2.82965
[300]	train-rmse:1.80459	eval-rmse:2.65936
Stopping. Best iteration:
[303]	train-rmse:1.79739	eval-rmse:2.65732

Best tree is 304, performance is 1.797392, 2.657317
[0]	train-rmse:8.69484	eval-rmse:9.35785
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 10 rounds.
[100]	train-rmse:3.81414	eval-rmse:4.66087
[200]	train-rmse:2.27655	eval-rmse:3.30779
[300]	train-rmse:1.81251	eval-rmse:2.95693
[400]	train-rmse:1.60783	eval-rmse:2.85228
[500]	train-rmse:1.47348	eval-rmse:2.81145
Stopping. Best iteration:
[549]	train-rmse:1.41809	eval-rmse:2.8001

Best tree is 550, performance is 1.418090, 2.800099
[0]	train-rmse:8.75882	eval-rmse:9.11317
Mult

In [21]:
n_splits=5
other_note ='_sick_test'
result['score']=result[['Score_%d'%i for i in range(1,n_splits+1)]].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
result.to_csv('../../Submission/result/result_%s'%today+other_note+'.csv',index=False)
submit['score'].to_csv('../../Submission/submit_%s'%today+other_note+'.csv',header=False,index=False)
test_result.to_csv('../../Submission/test/test_result_%s'%today+other_note+'.csv',index=False)
imp.to_csv('../../Submission/imp/importance_%s'%today+other_note+'.csv',index=False)

In [ ]:
###基础变量
1.83977998784

###增加部分比例数据，已经缺失体检项目情况
1.84295123211  imp来看，missing信息没有意义

###上一版去除missing
1.83839589522



###增加一些比值信息
1.84354856365


###根据日期的百分比，效果更加不好了
1.85046680299  变量选择之后 1.83862284729


###前面所有的特征
1.86013896413   变量选择之后  1.85446185976